## Loading data

In [1]:
import numpy as np
import pandas as pd
import geoplotlib as gp
from scipy import stats
from collections import Counter
import pickle
from __future__ import division
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder as LE

We start by reading in the data and dropping missing values from locations of the accidents.

In [2]:
df = pd.read_csv("NYPD_Motor_Vehicle_Collisions_reduced_data.csv", low_memory=False)

In [3]:
# Drop NaN values from Lat and Lon
df = df.dropna(subset=['LATITUDE','LONGITUDE', 'ON STREET NAME', 'BOROUGH'])

# Drop empty strings from street names
df = df[~df['ON STREET NAME'].str.contains('^\s+$')]

In order to keep the most relevant subset of the data, the accidents that occur at locations more than one standard deviation from the mean are discarded. Prior to this the LAT and LON were visualised and seemed somewhat normal distributed, which means that standard deviation is applicable in this instance.

In [4]:
df = df[(np.abs(stats.zscore(df[['LATITUDE','LONGITUDE']])) < 1).all(axis=1)]
df.shape

(656094, 37)

A quick preview of the data shows us how the current dataframe looks like.

In [5]:
df.head()

,Date,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,...,Maximum Humidity,Mean Temperature,Min Temperature,Minimum Humidity,Precipitation,Sea Level Pressure,Snow,Snow Depth,Visibility,Wind Speed
1,2017/03/28,0:00,BROOKLYN,11218,40.654080,-73.977610,"(40.65408, -73.97761)",18 STREET,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
4,2017/03/28,0:00,QUEENS,11429,40.713715,-73.731440,"(40.713715, -73.73144)",222 STREET,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
7,2017/03/28,0:01,BROOKLYN,11234,40.622400,-73.936646,"(40.6224, -73.936646)",FLATBUSH AVENUE,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
16,2017/03/28,10:02,BROOKLYN,11230,40.627130,-73.975280,"(40.62713, -73.97528)",ELMWOOD AVENUE,1,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
21,2017/03/28,10:16,BRONX,10455,40.814460,-73.896860,"(40.81446, -73.89686)",BRUCKNER BOULEVARD,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9


## Preprocessing data for modelling

### Speed limit parsing

An additional feature added to the data is speed limit of each street. The speed limit files were downloaded from http://www.nyc.gov/html/dot/html/about/vz_datafeeds.shtml. A dictionary is made (`speed_dict`) that will be composed of 5 keys and their valeus. Each key is a borough (e.g. Bronx) where the value is another dictionary of street:speed_limit for the streets in that borough.

In [6]:
speed_dict = {}
files = ['speed_limit_bronx.json', 'speed_limit_brooklyn.json', 'speed_limit_manhattan.json', 'speed_limit_queens.json', 'speed_limit_statenisland.json']

# go through all boroguh files
for file in files:
    temp_df = pd.read_json('speed_limits/' + file) # make a temporary dataframe
    temp_df.drop('type', inplace=True, axis=1) # drop the type column
    
    name = file.split('_')[-1].split('.')[0] # grab the borough name from the filename
    
    speed_dict[name] = {} # add the borough name to the speed dictionary

    # go through each street:speed_limit pair and add them to the current key in the dictionary
    for index, row in temp_df.iterrows():
        speed_dict[name][temp_df.iloc[index, 0]['properties']['street']] = temp_df.iloc[index, 0]['properties']['postvz_sl']

After creating the dictionary, a new empty column is added to the dataframe to hold the speed limits.

In [7]:
df['street_SL'] = np.zeros(df.shape[0])

Next, we go through all collisions and add speed limit on the street that they happened at to a new list (`speeds`), which then gets added as the (`street_SL`) column of our main dataframe.

In [8]:
boroughs = list(df['BOROUGH'].copy()) # list of all boroughs in the df
streets = list(df['ON STREET NAME'].copy()) # list of all street names in the df 
speeds = [] # a list of all speed limits

# cast to lowercase
boroughs = [x.lower().replace(" ", "") for x in boroughs]
streets = [x.strip() for x in streets]

# for each collision
for i in range(len(boroughs)):
    
    # try grabbing the relevant speed limit and add to the speeds list
    try:
        speed = speed_dict[boroughs[i]][streets[i]]
        speeds.append(speed)
    
    # if we can't find it, add NaN
    except KeyError:
        speeds.append(np.nan)

In [9]:
# cast to a numpy array and add to the dataframe
speeds = np.array(speeds)
df['street_SL'] = speeds

In [10]:
# finally, drop streets that no speed limit was found for
df = df.dropna(subset=['street_SL'])

### General parsing

Time is modeled as hour of the day.

In [11]:
# split at the ':' and grab the first value.
df['TIME'] = df['TIME'].apply(lambda x: int(x.split(':')[0]))

Next, create a new column for months only.

In [12]:
# split at '/' and grab the second value
df['Month'] = df['Date'].apply(lambda x: int(x.split('/')[1]))

Next, we use label encoder to numerically encode boroughs in NYC.

In [13]:
df_BR_dummies = pd.get_dummies(df['BOROUGH'])

Add the one-hot encoded columns to the dataframe.

In [14]:
df = df.join(df_BR_dummies)

Next, vehicle types will be encoded with one hot as well. First we will replace NaN values with 'OTHER'.

In [15]:
df[['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']] = df[['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']].fillna('OTHER')

Before the encoding is done, vehicles are grouped into 4 classes depending on their size:

In [16]:
name_dict = {
'two_wheeler': ['BICYCLE', 'PEDICAB', 'SCOOTER', 'MOTORCYCLE'],
'small': ['PASSENGER VEHICLE', 'TAXI'],
'medium': ['AMBULANCE', 'SPORT UTILITY / STATION WAGON', 'PICK-UP TRUCK', 'SMALL COM VEH(4 TIRES) ', 'LIVERY VEHICLE', 'VAN'],
'large': ['BUS', 'FIRE TRUCK', 'LARGE COM VEH(6 OR MORE TIRES)'],
'other': ['OTHER', 'UNKNOWN']}

And replace those values in the dictionary.

In [17]:
df_repl = df[['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']].copy()

for new_name in name_dict:
    
    for old_name in name_dict[new_name]:
    
        df_repl = df_repl[['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']].replace(to_replace=old_name, value=new_name)

Now we can apply one-hot endcoding to the vehicle type as well.

In [18]:
df_VT_dummies = pd.get_dummies(df_repl[['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']], prefix=['VTC1', 'VTC2'])

And we join that to the dataframe as well.

In [19]:
df = df.join(df_VT_dummies)

Lastly, we do one-hot encoding for events. First we start by replacing NaN with Other.

In [20]:
df['Events'] = df['Events'].fillna('Other')

Create a dummy dataframe to be added to the main dataframe later.

In [21]:
df_EV_dummies = pd.DataFrame(columns=['Rain', 'Snow', 'Fog', 'Other'], index=df.index)

Fill the dummy dataframe with empty values.

In [22]:
for column in df_EV_dummies.columns:
    df_EV_dummies[column] = np.zeros(df.shape[0])

Next, we go through the 'Events' column in the main dataframe (that contain values such as 'Fog\n\t,\nRain\n\t,\nSnow', split them and add corresponding '1's to the dummy dataframe for each event that happened during a given collision. For example, an 'Events' cell corresponding to 'Fog\n\t,\nRain\n\t,\nSnow' in the main dataframe would end up being

Rain | Snow | Fog | Other
--- | --- | --- |---
1 | 1 | 1 | 0

in the dummy events dataframe.

In [23]:
# loop through all rows in the dataframe
for index, row in enumerate(df['Events']):
    
    # for each event in the cell
    for event in row.split(','):
        
        # remove whitespace
        event = event.strip()
        
        # for each possible event
        for c_idx, column in enumerate(['Rain', 'Snow', 'Fog', 'Other']):
            
            # if it's found, fill the corresponding column in the dummies df
            if event == column:
                df_EV_dummies.iloc[index, c_idx] = 1

Rename columns because of same names in the main df, and remove the Other column as it does not provide valuable information.

In [24]:
df_EV_dummies.columns = ['Rain_EV', 'Snow_EV', 'Fog_EV', 'Other_EV']
df_EV_dummies = df_EV_dummies.drop('Other_EV', axis=1)

Now we can join the event dummy and main dataframes.

In [25]:
df = df.join(df_EV_dummies)

Originally, 'CONTRIBUTING FACTOR VEHICLE 1' was also modelled and encoded with one-hot encoding. However, not nearly all of the collisions have a contributing factor value and dropping them out significantly reduced the size of the data. After doing some ML modelling and performance tests with and without the 'CONTRIBUTING FACTOR VEHICLE 1', it was decided to not include the them as one-hot encoded values as the tradeoff is loss of data was not worth it.

There are some strange values in some of the rows still that should be numeric - for example in the 'Snow' column there are some rows with 'T' instead of snow depth. This also causes that column to be encoded as strings instead of numbers.

Therefore we need to cast the cells to numbers where we can.

In [26]:
Counter(df['Snow']).most_common(5)

[('0.00', 582842),
 ('T', 27057),
 ('1.02', 2204),
 ('0.25', 1846),
 ('1.27', 1730)]

Those that are encoded as strings:

- Max Gust Speed
- Max Wind Speed
- Precipitation
- Snow
- Snow Depth
- Wind Speed

In [27]:
def cast_to_float(x):
    
    try:
        return float(x)
        
    except Exception as error:
        return x

In [28]:
for column in ['Max Gust Speed', 'Max Wind Speed', 'Precipitation', 'Snow', 'Snow Depth', 'Wind Speed']:
    
    df[column] = df[column].apply(lambda x: cast_to_float(x))

Now we can select the relevant columns for the actual ML modelling:

In [29]:
df_ml = df.drop(['Date', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET NAME', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'UNIQUE KEY', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'Events', 'Dew Point'], axis=1)

Then, we can select those rows that only have numeric values.

In [30]:
df_ml = df_ml[df_ml.applymap(np.isreal).all(1)]

Now we need to add our y (any person injured or killed) to our dataframe. First we add an empty column to our dataframe.

In [31]:
df_ml['Y'] = np.zeros(df_ml.shape[0])

Then we look at the sum of people injured / killed in all the collisions. We sum over all the columns that keep track of casualties, and if are any casualties that is a positive observation (y = 1) and if there are non it is a negative observation (y = 0).

In [32]:
y = df_ml[['NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED']].sum(axis=1)

In [33]:
y = [1 if x > 0 else 0 for x in y]

In [34]:
df_ml['Y'] = y

Then of course we have to drop the columns that directly measure if people were killed or injured:

In [35]:
df_ml = df_ml.drop(['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 
                    'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
                   'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED'], axis=1)

Now we can save this parsed ML dataframe to a file.

In [36]:
df_ml.to_csv('NYPD_ML_df_wY.csv', index=False)

## Creating a balanced dataset

When evaluating ML models it is important to have balanced classes, as it can otherwise lead us to believe our model is doing very well when in fact it is just predicting all observations to be in a majority class. We start by reading in the dataframe we had saved earlier.

In [37]:
df_ml = pd.read_csv('NYPD_ML_df_wY.csv', low_memory=False)

There are still some NaN values in the dataset - we drop them before balancing.

In [38]:
df_ml = df_ml.dropna()

Let's look at the counts of our Y values.

In [39]:
Counter(df_ml['Y'])

Counter({0: 453018, 1: 107931})

We select our positive and negative classes from the dataframe.

In [40]:
pos_all = df_ml[df_ml['Y'] == 1]
neg_all = df_ml[df_ml['Y'] == 0]

Since the positive class is smaller than the negative class, we sample randomly from the negative class (without replacement), with the number of samples equaling the total amount of positive samples we have.

In [41]:
no_pos = pos_all.shape[0]
neg_sub = neg_all.sample(no_pos)

Lastly, we concatenate the equally sized positive and negative samples in a final dataframe, and save it as a .csv file.

In [42]:
df_comb = pd.concat([pos_all, neg_sub])

In [43]:
df_comb.to_csv('NYPD_ML_df_wY_balanced.csv', index=False)